# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [16]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [14]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)    
    

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


                    

## Creating Data Model based on Queries Above

In [8]:
# Drop table statement to be sure that is not created before. 
table_names = ['music_playlist_session', 'songs_user_session', 'names_library']

for table_name in table_names:
    try:
        drop_query = 'DROP TABLE IF EXISTS {}'.format(table_name)
        session.execute(drop_query)
        print("Table {} successfully dropped".format(table_name))
    except Exception as e:
        print(e)


Table music_playlist_session successfully dropped
Table songs_user_session successfully dropped
Table usernames_library successfully dropped


## Function to insert data into tables

In [18]:
def insert_data(query, session, values, dtypes, file):
    
    '''
    Pipeline to insert data to Apache Cassandra table from CSV File.
    Args:
        query (str): INSERT statement query
        session (object): Apache Cassandra Session
        values (list): list of indices to pass to INSERT statement
        dtype (list): list of values to convert `values`
        file (str): CSV file path
    '''
    # Validate that len values is equal to dtypes, if not, it's not possible to run the query.
    assert(len(values) == len(dtypes))
    # open csv file
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple([dtype(line[x])  for x, dtype in zip(values, dtypes)]))
            
file = 'event_datafile_new.csv'

## First Query
 - Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [11]:
#1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
#create table 
query_1 = "CREATE TABLE IF NOT EXISTS music_playlist_session"
query_1 = query_1 + "(session_id int, item_session int, user_id int, \
first_name varchar, last_name varchar, artist text, song varchar, length decimal, \
PRIMARY KEY(session_id, item_session))"

'''

We need the artist, song and length of every song. Filtered by session_id and item_session.
Based on those filters, we are going to create a composite primary key using session_id and item_session
(session_id, item_session). There's no need to order the output data
that's why we are not going to use any clustering key.

'''

try:
    session.execute(query_1)
except Exception as e:
    print(e)


In [12]:
###insert into music_playlist_session
# file name
file = 'event_datafile_new.csv'
#query
query = "INSERT INTO music_playlist_session (session_id, item_session, user_id, first_name, last_name, artist, song, length)"
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
# values and data types
values = [8,3,10,1,4,0,9,5]
####Using the float data type found that the effect is very poor, now replaced "Decimal".
dtypes = [int, int, int, str,str,str,str,Decimal]

insert_data(query, session, values, dtypes, file)

In [13]:
## Select statement to verify if the data was inserted into the table
try:
    rows = session.execute("SELECT * FROM music_playlist_session")
except Exception as e:
    print(e)
    
print("{} rows inserted".format(len(list(rows))))

6820 rows inserted


In [14]:
# Select First Query
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \ sessionId = 338, and itemInSession = 4
try:
    results_1 = session.execute("SELECT artist, song, length FROM music_playlist_session \
    WHERE session_id = 338 and item_session = 4")
except Exception as e:
    print(e)
    
for row in results_1:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Second Query
- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [34]:
# CREATE STATEMENT for second table based on second query.
query_2 = "CREATE TABLE IF NOT EXISTS songs_user_session"
query_2 = query_2 + "(user_id int, session_id int, item_session int,\
first_name text, last_name text, artist text, song text, \
PRIMARY KEY((user_id, session_id), item_session))"

try:
    session.execute(query_2)
except Exception as e:
    print(e)


## insert code for table songs_user_sessions

In [35]:
# insert into query
query = "INSERT INTO songs_user_session (user_id, session_id, item_session, first_name, last_name, artist, song)"
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

# values and dtypes

values = [10,8,3,1,4,0,9]
dtypes = [int, int, int, str, str, str, str]

insert_data(query, session, values, dtypes, file)

In [23]:
try:
    session.execute(create index songs_user_session_DateCreated on dotnetdemo.apilog (user_id, session_id);)
except Exception as e:
    print(e)

In [36]:
## Select statement to verify if the data was inserted into the table
try:
    rows = session.execute("SELECT artist, song, first_name, last_name  FROM songs_user_session\
    where user_id = 10 and session_id = 182")
except Exception as e:
    print(e)
    
print("{} rows inserted".format(len(list(rows))))

4 rows inserted


In [38]:
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

## Third Query
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### CREATE table

In [10]:
# CREATE STATEMENT for third table based on third query.
query_3 = "CREATE TABLE names_library"
query_3 = query_3 + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"

# We are going to use SONG and USER_ID as Primary Key and Clustering Key, this will allow the database to update the values
## which may be ok based on the statement of the query, because the team is asking about a list of users (name and last name)
### and not about how many times that song was listened.

try:
    session.execute(query_3)
except Exception as e:
    print(e)

In [20]:
# query
query = "INSERT INTO names_library (song, user_id, first_name, last_name)"
query = query + "VALUES (%s, %s, %s, %s)"
# values and dtypes
values = [9,10,1,4]
dtypes = [str, int, str, str]

# insert into table `usernames_library`
insert_data(query, session, values, dtypes, file)

In [22]:
# SELECT for third query
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    results_3 = session.execute("SELECT first_name, last_name FROM names_library \
    WHERE song = 'All Hands Against His Own'")
except Exception as e:
    print(e)
    
for row in results_3:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
## Drop table statements
table_names = ['music_playlist_session', 'songs_user_session', 'names_library']

for table_name in table_names:
    try:
        drop_query = 'DROP TABLE IF EXISTS {}'.format(table_name)
        session.execute(drop_query)
        print("Table {} successfully dropped".format(table_name))
    except Exception as e:
        print(e)

Table music_playlist_session successfully dropped
Table songs_user_session successfully dropped


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()